# Tempest Extreme experiment with NextGEMS data
Data have been already preprocessed to 1x1 grid on Levante

We start by reading and putting together into a single netcdf file

In [20]:
import xarray as xr
import os
import subprocess
from time import time
from glob import glob


def readwrite_from_lowres(filein, fileout) : 

    """
    Read and write low resolution data to mimic access from FDB

    Args: 
        filein: input file at low resolution
        fileout: input file at low resolution (netcdf)

    Returns: 
        outdict: dictionary with variable and dimensiona names for fileout
    """

    xfield = xr.open_mfdataset(filein)

    # check if output file exists
    if os.path.exists(fileout):
        os.remove(fileout)

    xfield.to_netcdf(fileout)
    xfield.close()
    
    outdict = {'lon': 'lon', 'lat': 'lat', 
            'psl': 'MSL', 'zg': 'Z',
            'uas': 'U10M', 'vas': 'V10M'}

    return outdict


def run_detect_nodes(tempest_dictionary, tempest_filein, tempest_fileout) : 

    """"
    Basic function to call from command line tempest extremes DetectNodes
    """
    
    detect_string= f'DetectNodes --in_data {tempest_filein} --timefilter 6hr --out {tempest_fileout} --searchbymin {tempest_dictionary["psl"]} ' \
    f'--closedcontourcmd {tempest_dictionary["psl"]},200.0,5.5,0;_DIFF({tempest_dictionary["zg"]}(30000Pa),{tempest_dictionary["zg"]}(50000Pa)),-58.8,6.5,1.0 --mergedist 6.0 ' \
    f'--outputcmd {tempest_dictionary["psl"]},min,0;_VECMAG({tempest_dictionary["uas"]},{tempest_dictionary["vas"]}),max,2 --latname {tempest_dictionary["lat"]} --lonname {tempest_dictionary["lon"]}'

    subprocess.run(detect_string.split(), stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

    return detect_string

def read_lonlat_nodes_old(tempest_fileout) :

    """
    Read from txt files output of DetectNodes the position of the centers of the TCs
    """

    # open the output file and extract the required lon/lat
    with open(tempest_fileout) as f:
       data = f.readlines()
    first = data[0].split('\t')
    date = first[0] + first[1].zfill(2) + first[2].zfill(2) + first[4].rstrip().zfill(2)
    slon =[]
    slat = []
    for line in data[1:] : 
        slon = slon + [line.split('\t')[3]]
        slat = slat + [line.split('\t')[4]]
    out = {'date': date, 'lon' : slon, 'lat' : slat}
    return out

def read_lonlat_nodes(tempest_fileout):
    """
    Read from txt files output of DetectNodes the position of the centers of the TCs
    """
    with open(tempest_fileout) as f:
        lines = f.readlines()
    first = lines[0].split('\t')
    date = first[0] + first[1].zfill(2) + first[2].zfill(2) + first[4].rstrip().zfill(2)
    lon_lat = [line.split('\t')[3:] for line in lines[1:]]
    out = {'date': date, 'lon': [val[0] for val in lon_lat], 'lat': [val[1] for val in lon_lat]}
    return out


def lonlatbox(lon, lat, delta) : 
    """
    Define the list for the box
    """
    return [float(lon) - delta, float(lon) +delta, float(lat) -delta, float(lat) + delta]

def store_fullres_field(mfield, xfield, nodes, boxdim = 10): 

    """Create xarray element that keep only the values of a field around the TC nodes"""

    mask = xfield * 0
    for k in range(0, len(nodes['lon'])) :
        # add safe condition: keep only data between 50S and 50N
        #if (float(nodes['lat'][k]) > -50) and (float(nodes['lat'][k]) < 50): 
        box = lonlatbox(nodes['lon'][k], nodes['lat'][k], boxdim)
        mask = mask + xr.where((xfield.lon > box[0]) & (xfield.lon < box[1]) & (xfield.lat > box[2]) & (xfield.lat < box[3]), True, False)

    outfield = xfield.where(mask>0)

    if isinstance(mfield, xr.DataArray):
        outfield = xr.concat([mfield, outfield], dim = 'time')
    
    return outfield

def write_fullres_field(gfield, filestore): 

    #compression = {'MSL': {'zlib': True}}
    compression = {'MSL': {'zlib': True, 'complevel': 1}}
    gfield.where(gfield!=0).to_netcdf(filestore, encoding=compression)
    gfield.close()

def clean_files(filelist):

    for fileout in filelist :
        if os.path.exists(fileout):
            os.remove(fileout)

def run_stitch_nodes(infiles_list, trackfile):

    full_nodes = os.path.join('full_nodes.txt')
    if os.path.exists(full_nodes):
            os.remove(full_nodes)

    with open(full_nodes, 'w') as outfile:
        for fname in sorted(infiles_list):
            with open(fname) as infile:
                outfile.write(infile.read())

    
    stitch_string = f'StitchNodes --in {full_nodes} --out {trackfile} --in_fmt lon,lat,slp,wind --range 8.0 --mintime 54h ' \
        f'--maxgap 24h --threshold wind,>=,10.0,10;lat,<=,50.0,10;lat,>=,-50.0,10'
    subprocess.run(stitch_string.split())
    return stitch_string


In [ ]:
# pa# input directory
indir='/home/b/b382216/scratch/regrid'
tmpdir='/home/b/b382216/scratch/tmpdir'
fulldir='/home/b/b382216/scratch/fullres'

time = '000120'
infile = os.path.join(indir, f'regrid+{time}_*.nc')
outfile = os.path.join(tmpdir, time + '.nc')
txtfile = os.path.join(tmpdir, 'output_' + time + '.txt')
mslfile=os.path.join(fulldir, f'ICMGGhqys+{time}_msl.nc')
storefile = os.path.join(tmpdir, 'TC_' + time + '.nc')

# read and save the netcdf file to mimic the time lost by the FDB query
tempest_dictionary = readwrite_from_lowres(infile, outfile)
tempest_command = run_detect_nodes(tempest_dictionary, outfile, txtfile)
tempest_nodes = read_lonlat_nodes(txtfile) 

xfield = xr.open_mfdataset(outfile)['MSL']

gfield = xfield * 0
for k in range(0, len(tempest_nodes['lon'])) :
    box = lonlatbox(tempest_nodes['lon'][k], tempest_nodes['lat'][k], 10)
    gfield = gfield + xr.where((xfield.lon > box[0]) & (xfield.lon < box[1]) & (xfield.lat > box[2]) & (xfield.lat < box[3]), True, False)

xfield = xfield.where(gfield>0)

xfield.plot()

In [21]:

# input directory
indir='/home/b/b382216/scratch/regrid'
tmpdir='/home/b/b382216/scratch/tmpdir'
fulldir='/home/b/b382216/scratch/fullres'
#dask.config.set(scheduler="synchronous")

# loop on timerecors
for t in range(6000, 12000, 120) : 
    tic = time()

    tttt = str(t).zfill(6)
    print(tttt)
    # path definition
    original_file = os.path.join(indir, f'regrid+{tttt}_*.nc')
    netcdf_file = os.path.join(tmpdir, tttt + '.nc')
    txt_file = os.path.join(tmpdir, 'output_' + tttt + '.txt')
    msl_fullres_file=os.path.join(fulldir, f'ICMGGhqys+{tttt}_msl.nc')
   

    # read and save the netcdf file to mimic the time lost by the FDB query
    tempest_dictionary = readwrite_from_lowres(original_file, netcdf_file)

    # run tempest extremes
    tempest_command = run_detect_nodes(tempest_dictionary, netcdf_file, txt_file)
    tempest_nodes = read_lonlat_nodes(txt_file)

    # get the full res field and store the required values around the Nodes
    fullres_field = xr.open_mfdataset(msl_fullres_file)['MSL']
    xfield = store_fullres_field(0, fullres_field, tempest_nodes)
    clean_files([netcdf_file])
    
    toc = time()
    print('Timestep done in {:.4f} seconds'.format(toc - tic))

    # store data every day
    #if (t+120) % (120*4) == 0 : 
        #day = str((t+120) // (120*4)).zfill(3)
    id = tempest_nodes['date']
    print(id)
    print('Storing output')
    tic = time()

    # store the file
    store_file = os.path.join(tmpdir, f'TC_MSL_{id}.nc')
    write_fullres_field(xfield, store_file)

    ##reinit xfield
    #xfield = 0

    toc = time()
    print('Storing timestep data done in {:.4f} seconds'.format(toc - tic))


006000
Timestep done in 0.5145 seconds
2020020112
Storing output
Storing timestep data done in 0.9771 seconds
006120
Timestep done in 0.4299 seconds
2020020118
Storing output
Storing timestep data done in 1.2800 seconds
006240
Timestep done in 0.2842 seconds
2020020200
Storing output
Storing timestep data done in 1.0040 seconds
006360
Timestep done in 0.5731 seconds
2020020206
Storing output
Storing timestep data done in 1.1940 seconds
006480
Timestep done in 0.3484 seconds
2020020212
Storing output
Storing timestep data done in 1.0531 seconds
006600
Timestep done in 0.7050 seconds
2020020218
Storing output
Storing timestep data done in 1.5299 seconds
006720
Timestep done in 0.4912 seconds
2020020300
Storing output
Storing timestep data done in 1.2131 seconds
006840
Timestep done in 0.5674 seconds
2020020306
Storing output
Storing timestep data done in 1.3032 seconds
006960
Timestep done in 0.6927 seconds
2020020312
Storing output


PermissionError: [Errno 13] Permission denied: b'/home/b/b382216/scratch/tmpdir/TC_MSL_2020020312.nc'

In [19]:

track_file = os.path.join(tmpdir, 'track.txt')
filenames = glob(os.path.join(tmpdir,'output*.txt'))
stitch_string = run_stitch_nodes(filenames, track_file)


    


TypeError: 'module' object is not callable

Get the lon/lat from the txt file

In [9]:
from glob import glob








StitchNodes --in /home/b/b382216/scratch/tmpdir/full_nodes.txt --out /home/b/b382216/scratch/tmpdir/track.txt --in_fmt lon,lat,slp,wind --range 8.0 --mintime 54h --maxgap 24h --threshold wind,>=,10.0,10;lat,<=,50.0,10;lat,>=,-50.0,10
Arguments:
  --in <string> ["/home/b/b382216/scratch/tmpdir/full_nodes.txt"] 
  --in_list <string> [""] 
  --in_connect <string> [""] 
  --out <string> ["/home/b/b382216/scratch/tmpdir/track.txt"] 
  --in_fmt <string> ["lon,lat,slp,wind"] 
  --range <double> [8.000000] (degrees)
  --mintime <string> ["54h"] 
  --time_begin <string> [""] 
  --time_end <string> [""] 
  --min_endpoint_dist <double> [0.000000] (degrees)
  --min_path_dist <double> [0.000000] (degrees)
  --maxgap <string> ["24h"] 
  --threshold <string> ["wind,>=,10.0,10;lat,<=,50.0,10;lat,>=,-50.0,10"] [col,op,value,count;...]
  --caltype <string> ["standard"] (none|standard|noleap|360_day)
  --allow_repeated_times <bool> [false] 
  --out_file_format <string> ["gfdl"] (gfdl|csv|csvnohead)
-----

CompletedProcess(args=['StitchNodes', '--in', '/home/b/b382216/scratch/tmpdir/full_nodes.txt', '--out', '/home/b/b382216/scratch/tmpdir/track.txt', '--in_fmt', 'lon,lat,slp,wind', '--range', '8.0', '--mintime', '54h', '--maxgap', '24h', '--threshold', 'wind,>=,10.0,10;lat,<=,50.0,10;lat,>=,-50.0,10'], returncode=0)

In [16]:
 # open the output file and extract the required lon/lat
track_file = os.path.join(tmpdir, 'track.txt')


with open(track_file) as file:
    lines = file.read().splitlines()
    parts_list = [line.split("\t") for line in lines if len(line.split("\t")) > 6]
    #print(parts_list)
    tracks ={'slon': [parts[3] for parts in parts_list],
        'slat':  [parts[4] for parts in parts_list],
        'date': [parts[7] + parts[8].zfill(2) + parts[9].zfill(2) + parts[10].zfill(2) for parts in parts_list],
    }

reorder_track ={}
for tstep in sorted(set(tracks['date'])) : 
    #idx = tracks['date'].index(tstep)
    idx = [i for i, e in enumerate(tracks['date']) if e == tstep]
    reorder_track[tstep] = {}
    reorder_track[tstep]['lon'] = [tracks['slon'][k] for k in idx]
    reorder_track[tstep]['lat'] = [tracks['slat'][k] for k in idx]



xfield = 0
for id in reorder_track.keys():

    print(id)
    fullres_file = os.path.join(tmpdir, f'TC_MSL_{id}.nc')
    fullres_field = xr.open_mfdataset(fullres_file)['MSL']

    # get the full res field and store the required values around the Nodes
    #fullres_field = xr.open_mfdataset(msl_fullres_file)['MSL']
    xfield = store_fullres_field(xfield, fullres_field, tempest_nodes)


print('Storing output')
tic = time()

# store the file
store_file = os.path.join(tmpdir, 'tempest_tracks.nc')
write_fullres_field(xfield, store_file)
toc = time()
print('Storing timestep data done in {:.4f} seconds'.format(toc - tic))






2020020312
2020020318
2020020400
2020020406
2020020412
2020020418
2020020500
2020020506
2020020512
2020020518
2020020600
2020020606
2020020612
2020020618
2020020700
2020020706
2020020712
2020020718
2020020800
2020020806
2020020812
2020020818
2020020900
2020020906
2020020912
2020020918
2020021000
2020021006
2020021012
2020021018
2020021100
2020021106
2020021112
2020021118
2020021200
2020021206
2020021212
2020021218
2020021300
2020021306
2020021312
2020021318
Storing output
Storing timestep data done in 133.7639 seconds


In [ ]:
# Use this diagnostic to test the new aqua.docker.rundiag() function
# Everything is defined in the diagnostic.yaml and machine.yaml files
# If no argument is provided then the first command defined in machine.yaml is run
#output = aqua.docker.rundiag(cmd="detect")

# replace the dokker structure with a python environment which does the following:

# detect_out = aqua.docker.rundiag(cmd="detect")
# stitch_out = aqua.docker.rundiag(cmd="stitch")

import sys
sys.path.append("../..")
from aqua import regrid

import os
import subprocess
import numpy as np
import re
import yaml
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

from cartopy import config
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

#import tempest_lib as tmp


diagname = 'TCs'
machine = 'levante'


with open(f'../../AQUA/diagnostics/config/config_{machine}.yml', 'r', encoding='utf-8') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

with open(f'{diagname}.yml', 'r', encoding='utf-8') as file:
    namelist = yaml.load(file, Loader=yaml.FullLoader)

#regrid files from original res to 1x1

from cdo import *   # python version
cdo = Cdo()

k_init=000000
k_final=166560
#loop to regrid files 
for k in range(k_init,k_final):

        print("GH regridding")
        cdo genbil,r360x180 file_in weights.nc
        cdo remap, r360x180, weights file_in file_out

        print("lnsp regridding")
        print("10u regridding")
        print("10v regridding")


# ADJUST TEMPEST COMMANDS IN THE RESPECTIVE YML FILES

# first run detect_nodes to detect the TCs centers
subprocess.run(['bash', './detect_nodes'])

# then compute trajectories using stitch_nodes
subprocess.run(['bash', './stitch_nodes'])

In [ ]:
# from https://github.com/zarzycki/cymep

def getTrajectories(filename,nVars,headerDelimStr,isUnstruc):
  print("Getting trajectories from TempestExtremes file...")
  print("Running getTrajectories on '%s' with unstruc set to '%s'" % (filename, isUnstruc))
  print("nVars set to %d and headerDelimStr set to '%s'" % (nVars, headerDelimStr))

  # Using the newer with construct to close the file automatically.
  with open(filename) as f:
      data = f.readlines()

  # Find total number of trajectories and maximum length of trajectories
  numtraj=0
  numPts=[]
  for line in data:
    if headerDelimStr in line:
      # if header line, store number of points in given traj in numPts
      headArr = line.split()
      numtraj += 1
      numPts.append(int(headArr[1]))
    else:
      # if not a header line, and nVars = -1, find number of columns in data point
      if nVars < 0:
        nVars=len(line.split())
  
  maxNumPts = max(numPts) # Maximum length of ANY trajectory

  print("Found %d columns" % nVars)
  print("Found %d trajectories" % numtraj)

  # Initialize storm and line counter
  stormID=-1
  lineOfTraj=-1

  # Create array for data
  if isUnstruc:
    prodata = np.empty((nVars+1,numtraj,maxNumPts))
  else:
    prodata = np.empty((nVars,numtraj,maxNumPts))

  prodata[:] = np.NAN

  for i, line in enumerate(data):
    if headerDelimStr in line:  # check if header string is satisfied
      stormID += 1      # increment storm
      lineOfTraj = 0    # reset trajectory line to zero
    else:
      ptArr = line.split()
      for jj in range(nVars):
        if isUnstruc:
          prodata[jj+1,stormID,lineOfTraj]=ptArr[jj]
        else:
          prodata[jj,stormID,lineOfTraj]=ptArr[jj]
      lineOfTraj += 1   # increment line

  print("... done reading data")
  return numtraj, maxNumPts, prodata


def getNodes(filename,nVars,isUnstruc):
  print("Getting nodes from TempestExtremes file...")

  # Using the newer with construct to close the file automatically.
  with open(filename) as f:
      data = f.readlines()

  # Find total number of trajectories and maximum length of trajectories
  numnodetimes=0
  numPts=[]
  for line in data:
    if re.match(r'\w', line):
      # if header line, store number of points in given traj in numPts
      headArr = line.split()
      numnodetimes += 1
      numPts.append(int(headArr[3]))
    else:
      # if not a header line, and nVars = -1, find number of columns in data point
      if nVars < 0:
        nVars=len(line.split())
  
  maxNumPts = max(numPts) # Maximum length of ANY trajectory

  print("Found %d columns" % nVars)
  print("Found %d trajectories" % numnodetimes)
  print("Found %d maxNumPts" % maxNumPts)

  # Initialize storm and line counter
  stormID=-1
  lineOfTraj=-1

  # Create array for data
  if isUnstruc:
    prodata = np.empty((nVars+5,numnodetimes,maxNumPts))
  else:
    prodata = np.empty((nVars+4,numnodetimes,maxNumPts))

  prodata[:] = np.NAN

  nextHeadLine=0
  for i, line in enumerate(data):
    if re.match(r'\w', line):  # check if header string is satisfied
      stormID += 1      # increment storm
      lineOfTraj = 0    # reset trajectory line to zero
      headArr = line.split()
      YYYY = int(headArr[0])
      MM = int(headArr[1])
      DD = int(headArr[2])
      HH = int(headArr[4])
    else:
      ptArr = line.split()
      for jj in range(nVars-1):
        if isUnstruc:
          prodata[jj+1,stormID,lineOfTraj]=ptArr[jj]
        else:
          prodata[jj,stormID,lineOfTraj]=ptArr[jj]
      if isUnstruc:
        prodata[nVars+1,stormID,lineOfTraj]=YYYY
        prodata[nVars+2,stormID,lineOfTraj]=MM
        prodata[nVars+3,stormID,lineOfTraj]=DD
        prodata[nVars+4,stormID,lineOfTraj]=HH
      else:
        prodata[nVars  ,stormID,lineOfTraj]=YYYY
        prodata[nVars+1,stormID,lineOfTraj]=MM
        prodata[nVars+2,stormID,lineOfTraj]=DD
        prodata[nVars+3,stormID,lineOfTraj]=HH
      lineOfTraj += 1   # increment line

  print("... done reading data")
  return numnodetimes, maxNumPts, prodata

In [ ]:
# tempest settings
nVars=-1
headerStr='start'
isUnstruc = 0

# Extract trajectories from tempest file and assign to arrays
# USER_MODIFY
nstorms, ntimes, traj_data = getTrajectories(trajfile,nVars,headerStr,isUnstruc)
xlon   = traj_data[2,:,:]
xlat   = traj_data[3,:,:]
xpres  = traj_data[4,:,:]/100.
xwind  = traj_data[5,:,:]
xyear  = traj_data[7,:,:]
xmonth = traj_data[8,:,:]

# Initialize axes
ax = plt.axes(projection=ccrs.PlateCarree())
# ax.set_extent([-180, 180, -75, 75], crs=None)

# Set title and subtitle
plt.title('Example of a Trajectory Plot')


# Set land feature and change color to 'lightgrey'
# See link for extensive list of colors:
# https://matplotlib.org/3.1.0/gallery/color/named_colors.html
ax.add_feature(cfeature.LAND, color='lightgrey')

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=0.5, color='k', alpha=0.5, linestyle='--')
gl.xlabels_top = False
gl.ylabels_left = False
#gl.xlines = False
gl.xlocator = mticker.FixedLocator([-180, -90, 0, 90, 180])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.ylabel_style = {'size': 12, 'color': 'black'}
gl.xlabel_style = {'size': 12, 'color': 'black'}



# Plot each trajectory
for i in range(nstorms):

        # doesn't work with cartopy!
        #plt.plot(xlon[i], xlat[i], linewidth=0.4)



    plt.scatter(x=xlon[i], y=xlat[i],
                                                color="black",
                                                s=30,
                                                linewidths=0.5,
                                                marker=".",
                                                alpha=0.8,
                                                transform=ccrs.PlateCarree()) ## Important


plt.savefig(plotdir + "prova_TC_2010.png", bbox_inches='tight', dpi=350)